# Predicting Box Office Revenue

In [69]:
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

### Data cleanup

Join to two dataset files by movie id, and drop duplicate columns.

In [70]:
movies_data = pd.read_csv('../tmdb_5000_movies.csv')
credits_data = pd.read_csv('../tmdb_5000_credits.csv')

# Perform an inner join on the movies and credits data so it's all in one dataframe
all_movies_data = pd.merge(movies_data, credits_data, left_on='id', right_on='movie_id', how='inner', suffixes=('_movies', '_credits'))

# Drop duplicate id and title fields
all_movies_data = all_movies_data.drop(columns=['movie_id', 'title_credits'])
# Rename original title field
all_movies_data = all_movies_data.rename(columns={'title_movies': 'title'})

print('Movies remaining in dataset after merge: {}'.format(len(all_movies_data)))

Movies remaining in dataset after merge: 4803


Drop rows with no budget or revenue data, since that's what we want to analyze. 

In [71]:
# Remove data with no revenue or budget info
all_movies_data = all_movies_data[all_movies_data['budget'] > 0]
all_movies_data = all_movies_data[all_movies_data['revenue'] > 0]

print('Movies remaining in dataset after dropping rows without budget or revenue data: {}'.format(len(all_movies_data)))

Movies remaining in dataset after dropping rows without budget or revenue data: 3229


Convert the genre field from a string to a list of dictionaries. Drop rows that cannot be formatted.

In [84]:
def format_genres(genre_list):
    try:
        return json.loads(genre_list)
    except (ValueError, SyntaxError, TypeError):
        return np.nan

# Avoiding in-place modification here so the cells run smoothly
all_movies_data['formatted_genres'] = all_movies_data['genres'].apply(format_genres)
all_movies_data = all_movies_data[all_movies_data['formatted_genres'] != np.nan]

print('Formatted genre field: {}'.format(all_movies_data.iloc[0]['formatted_genres']))
print('Movies remaining in dataset after formatting genres: {}'.format(len(all_movies_data)))

Formatted genre field: [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]
Movies remaining in dataset after formatting genres: 3229


### Descriptive Statistics

#### Revenue - general descriptive statistics

In [73]:
all_movies_revenue = all_movies_data['revenue']
mean_all_movies_revenue = np.mean(all_movies_revenue)
median_all_movies_revenue = np.median(all_movies_revenue)
std_all_movies_revenue = np.std(all_movies_revenue)

all_movies_data[['revenue']].describe().style.format("{:,.2f}")

,revenue
count,"3,229.00"
mean,"121,242,957.26"
std,"186,302,864.01"
min,5.00
25%,"17,000,000.00"
50%,"55,184,721.00"
75%,"146,292,009.00"
max,"2,787,965,087.00"


#### Budget - general descriptive statistics

In [74]:
all_movies_budget = all_movies_data['budget']
mean_all_movies_revenue = np.mean(all_movies_budget)
median_all_movies_revenue = np.median(all_movies_budget)
std_all_movies_revenue = np.std(all_movies_budget)

all_movies_data[['budget']].describe().style.format("{:,.2f}")

,budget
count,"3,229.00"
mean,"40,654,444.77"
std,"44,396,741.80"
min,1.00
25%,"10,500,000.00"
50%,"25,000,000.00"
75%,"55,000,000.00"
max,"380,000,000.00"


#### Revenue:Budget Profit Ratio - general descriptive statistics

In [75]:
all_movies_data['profit_ratio'] = all_movies_data['revenue'] / all_movies_data['budget']

all_movies_profit_ratio = all_movies_data['profit_ratio']
mean_all_movies_revenue = np.mean(all_movies_profit_ratio)
median_all_movies_revenue = np.median(all_movies_profit_ratio)
std_all_movies_revenue = np.std(all_movies_profit_ratio)

all_movies_data[['profit_ratio']].describe().style.format("{:,.2f}")

,profit_ratio
count,"3,229.00"
mean,"2,954.82"
std,"150,610.10"
min,0.00
25%,1.02
50%,2.30
75%,4.42
max,"8,500,000.00"
